In [10]:
import numpy as np
import cv2
from vidgear.gears import VideoGear

In [11]:
VIDEO = "video/object_detection_road.mp4"
BORDER_INCREASE = 0.12

In [12]:
#Stabilise video feed
vidcap = VideoGear(source=VIDEO, stabilize = True).start()

frame1 = vidcap.read()
frame2 = vidcap.read()

while frame2 is not None:
    #Find difference between two frames
    frame1 = np.ascontiguousarray(frame1, dtype=np.uint8)
    frame2 = np.ascontiguousarray(frame2, dtype=np.uint8)
    
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    
    #Irrelevant parts of frame blocked out
    #Top of frame, stabilisation makes it shakey and unreliable for detection
    cv2.rectangle(gray, (0, 0), (1920, 50), (0,0,0), -1)
    
    #Trees on left side
    vertices = np.array([[0, 1080], [0, 0], [1075, 0], [300,1080]], np.int32)
    pts = vertices.reshape((-1, 1, 2))
    cv2.polylines(gray, [pts], isClosed=True, color=(0, 0, 0), thickness=20)
    cv2.fillPoly(gray, [pts], color=(0, 0, 0))
    
    #Trees on right side
    vertices = np.array([[1920, 1080], [1920, 0], [1250, 0]], np.int32)
    pts = vertices.reshape((-1, 1, 2))
    cv2.polylines(gray, [pts], isClosed=True, color=(0, 0, 0), thickness=20)
    cv2.fillPoly(gray, [pts], color=(0, 0, 0))
    
    #Outline areas of movement
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 40, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations = 3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #Multiple contours are produced which may overlapped each other or belong to the same vehicle
    #Create a heatmap of these areas to produce a single contour for each area of movement
    AoI = np.ascontiguousarray(np.zeros(shape=frame1.shape), dtype=np.uint8)

    for con in contours:
        (x, y, w, h) = cv2.boundingRect(con)
        
        #Ignore contour if it is too small
        if cv2.contourArea(con) < 500:
            continue
        
        #Increase the size of each contour incase it is too small
        h_diff = int(w*BORDER_INCREASE)
        w_diff = int(h*BORDER_INCREASE)

        cv2.rectangle(AoI, (x-h_diff, y-w_diff), (x+w+h_diff, y+h+w_diff), (255,255,255), -1)
    
    #create new contours for these areas and display them on first frame
    AoI = cv2.cvtColor(AoI, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(AoI, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for con in contours:
        (x, y, w, h) = cv2.boundingRect(con)
        
        if cv2.contourArea(con) < 500:
            continue

        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0,0,0), 2)
    
    #Show frame
    cv2.imshow('Frame',frame1)
    frame1 = frame2
    
    #Skip next two frames to allow for better detection of movement
    for x in range(2):
        frame2 = vidcap.read()
    
    if frame2 is not None:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
    
# Closes all the frames
cv2.destroyAllWindows()